# TP53

In [ ]:
import pandas as pd
TP53_ClinVar = pd.read_csv("/gpfs/home/pl2948/VariantInterpretation/Data/TP53_ESM_DMS.csv")
TP53_ClinVar

In [ ]:
import numpy as np

feature_column = ['ESM1b_score',
                  'TP53_transcription_urn_mavedb_00001234-0-1_scores',
                  'TP53_DNE_urn_mavedb_00001235-a-1_scores',
                  'urn_mavedb_00000068-b-1_scores',
                  'urn_mavedb_00001234-g-1_scores',
                  'urn_mavedb_00001234-e-1_scores', 
                  'urn_mavedb_00001234-d-1_scores',
                  'urn_mavedb_00001213-a-1_scores',
                  'urn_mavedb_00001236-0-1_scores', 
                  'urn_mavedb_00001234-h-1_scores',
                  'urn_mavedb_00000068-c-1_scores', 
                  'urn_mavedb_00001234-a-1_scores',
                  'urn_mavedb_00001234-c-1_scores', 
                  'urn_mavedb_00001234-b-1_scores',
                  'urn_mavedb_00001234-f-1_scores', 
                  'urn_mavedb_00000068-0-1_scores',
                  'urn_mavedb_00000068-a-1_scores']

feature = TP53_ClinVar[feature_column].to_numpy().astype(float)
print(feature.shape)

label = TP53_ClinVar['ClinVar_annotation'].to_numpy().astype(int)
print(label.shape)

In [ ]:
TP53_ClinVar[feature_column].isna().sum()

## Tool exploration

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

corr_matrix = np.abs(TP53_ClinVar[feature_column].corr(method='pearson'))

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", square=True,
            cbar_kws={"shrink": .75}, linewidths=0.5, linecolor='white')
plt.title("Pearson Correlation Heatmap between MAVEs Scores")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
from sklearn.feature_selection import mutual_info_regression

def conditional_mutual_info(feature_1, feature_2, label):
    feature_1, feature_2 = feature_1.reshape(-1, 1), feature_2.reshape(-1, 1)
    
    feature_1_0, feature_2_0 = feature_1[label == 0], feature_2[label == 0]
    feature_1_1, feature_2_1 = feature_1[label == 1], feature_2[label == 1]

    if len(feature_1_0) > 1 and len(feature_2_0) > 1:
        mi_0 = mutual_info_regression(feature_1_0, feature_2_0.ravel(), random_state=0)[0]
    else:
        mi_0 = 0  

    if len(feature_1_1) > 1 and len(feature_2_1) > 1:
        mi_1 = mutual_info_regression(feature_1_1, feature_2_1.ravel(), random_state=0)[0]
    else:
        mi_1 = 0 

    p0, p1 = np.mean(label == 0), np.mean(label == 1)
    conditional_mi = p0 * mi_0 + p1 * mi_1

    return conditional_mi

In [ ]:
import numpy as np

def permutation_pvalue_cmi(feature_1, feature_2, label, n_perm=10000, random_state=0):
    rng = np.random.default_rng(random_state)

    I_obs = conditional_mutual_info(feature_1, feature_2, label)

    unique_labels = np.unique(label)
    group_indices = {lv: np.where(label == lv)[0] for lv in unique_labels}

    exceed = 0
    for _ in range(n_perm):
        feature_2_perm = np.array(feature_2, copy=True)
        # re assign feature_2
        for idx in group_indices.values():
            feature_2_perm[idx] = feature_2_perm[idx][rng.permutation(len(idx))]
        I_perm = conditional_mutual_info(feature_1, feature_2_perm, label)
        if I_perm >= I_obs - 1e-12:
            exceed += 1

    p_val = (exceed + 1) / (n_perm + 1)
    return I_obs, p_val

In [ ]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import roc_auc_score
from P_KNN_GPU import silhouette_score_1d_torch

for i in range(1,len(feature_column)):
    valid_label = label[~np.isnan(feature[:,i])].ravel()
    valid_feature = feature[~np.isnan(feature[:,i]), i]
    ESM = feature[~np.isnan(feature[:,i]), 0]
    
    roc_auc = roc_auc_score(valid_label, valid_feature)
    if roc_auc < 0.5: roc_auc = 1-roc_auc
    print(f"ROC-AUC: {roc_auc:.4f}")

    mutual_information = mutual_info_classif(valid_feature.reshape(-1, 1), valid_label)
    print(f"Mutual Information: {mutual_information[0]:.4f}")

    silhouette_score= silhouette_score_1d_torch(torch.tensor(valid_feature).reshape(-1, 1), torch.tensor(valid_label))
    print(f"Silhouette score: {silhouette_score:.4f}")

    # conditional_mi = conditional_mutual_info(ESM, valid_feature, valid_label)
    
    conditional_mi, p_val = permutation_pvalue_cmi(ESM, valid_feature, valid_label, n_perm=1000, random_state=0)
    print(f"Conditional Mutual Information: {conditional_mi:.4f}, p={p_val:.4f}")

    rank_valid_feature = np.argsort(np.argsort(valid_feature))
    rank_ESM = np.argsort(np.argsort(ESM))
    
    plt.figure(figsize=(6, 6))

    plt.hist(feature[label == 0, i], color='blue', alpha=0.7, label="Benign")
    plt.hist(feature[label == 1, i], color='red', alpha=0.7, label="Pathogenic")
    
    plt.xlabel("ESM1b")
    plt.ylabel("Frequency")
    plt.title(feature_column[i])
    plt.legend()
    # plt.savefig("Fig6BESM1b.svg", format="svg")
    plt.show()

    fig, axs = plt.subplots(1, 2, figsize=(12, 6), constrained_layout=True)
    
    fig.suptitle(feature_column[i])
    
    axs[0].scatter(valid_feature[valid_label==0], ESM[valid_label==0], color='blue', alpha=0.7, label='Benign')
    axs[0].scatter(valid_feature[valid_label==1], ESM[valid_label==1], color='red', alpha=0.7, label='Pathogenic')
    axs[0].set_xlabel("MAVE score", fontsize=12)
    axs[0].set_ylabel("ESM1b Score", fontsize=12)
    axs[0].legend()
    
    axs[1].scatter(rank_valid_feature[valid_label==0], rank_ESM[valid_label==0], color='blue', alpha=0.7)
    axs[1].scatter(rank_valid_feature[valid_label==1], rank_ESM[valid_label==1], color='red', alpha=0.7)
    axs[1].set_xlabel("MAVE rank score", fontsize=12)
    axs[1].set_ylabel("ESM1b rank Score", fontsize=12)
    
    # plt.savefig("Fig6_combined.svg", format="svg")
    plt.show()

## Calibration

In [ ]:
import numpy as np

feature_column = ['ESM1b_score',
                  'TP53_transcription_urn_mavedb_00001234-0-1_scores',
                  'TP53_DNE_urn_mavedb_00001235-a-1_scores',
                  'urn_mavedb_00000068-b-1_scores',
                  'urn_mavedb_00001234-g-1_scores',
                  'urn_mavedb_00001234-e-1_scores', 
                  'urn_mavedb_00001234-d-1_scores',
                  'urn_mavedb_00001213-a-1_scores',
                  'urn_mavedb_00001236-0-1_scores', 
                  'urn_mavedb_00001234-h-1_scores',
                  'urn_mavedb_00000068-c-1_scores', 
                  'urn_mavedb_00001234-a-1_scores',
                  'urn_mavedb_00001234-c-1_scores', 
                  'urn_mavedb_00001234-b-1_scores',
                  'urn_mavedb_00001234-f-1_scores', 
                  'urn_mavedb_00000068-0-1_scores',
                  'urn_mavedb_00000068-a-1_scores']

feature = TP53_ClinVar[feature_column].to_numpy().astype(float)
print(feature.shape)

label = TP53_ClinVar['ClinVar_annotation'].to_numpy().astype(int)
print(label.shape)

In [ ]:
Pprior = 0.22 # Prior probability of pathogenicity (changes w/ c)
n_calibration_in_window = 10;   # minimum number of clinvar variants in a local window
frac_regularization_in_window=0.03
w_calibration=None
batch_size=512
n_bootstrap = 100

p_value = 0.05
logbase = 1730

In [ ]:
import numpy as np
import torch
from P_KNN_GPU import get_bootstrap_KNN_score_gpu
import gc

for tool in range(len(feature_column)):
    print(f"MAVE: {feature_column[tool]}")
    valid_label = label[~np.isnan(feature[:,tool])]
    valid_feature = feature[~np.isnan(feature[:,tool]), tool]
    
    test_results_array = np.zeros((len(valid_feature), n_bootstrap))

    normalization='z'
    impute=False
    mi_scaling=False
    
    for i in range(len(valid_feature)):
        test_feature = valid_feature[i].reshape(-1, 1)
        test_label = np.array([valid_label[i]])
    
        feature_c = np.delete(valid_feature, i, axis=0).reshape(-1, 1)
        label_c = np.delete(valid_label, i, axis=0)

        torch.cuda.empty_cache()
        gc.collect()
        
        test_results = get_bootstrap_KNN_score_gpu(feature_c, test_feature, feature_c, 
                                                   label_c, Pprior, w_calibration, 
                                                   n_calibration_in_window, frac_regularization_in_window, 
                                                   normalization, 
                                                   impute, 
                                                   mi_scaling, 
                                                   n_bootstrap, batch_size)
    
        test_results_array[i, :] = test_results

    np.save(f'/gpfs/home/pl2948/VariantInterpretation/KNNTestResultArrays/KNN_MAVE_{feature_column[tool]}.npy', test_results_array)
    print('Single Tool calibration completed')

    if tool == 0: continue
    
    ESM_feature = feature[:, [0, tool]] 
    test_results_array = np.zeros((len(ESM_feature), n_bootstrap))

    normalization='rank' 
    impute=True
    mi_scaling=True

    for i in range(len(ESM_feature)):
        test_feature = ESM_feature[i].reshape(1, -1)
        test_label = np.array([label[i]]) 
        
        feature_c = np.delete(ESM_feature, i, axis=0)
        label_c = np.delete(label, i, axis=0)
    
        test_results = get_bootstrap_KNN_score_gpu(feature_c, test_feature, feature_c, 
                                                   label_c, Pprior, w_calibration, 
                                                   n_calibration_in_window, frac_regularization_in_window, 
                                                   normalization, 
                                                   impute, 
                                                   mi_scaling, 
                                                   n_bootstrap, batch_size)

        test_results_array[i, :] = test_results

    np.save(f'/gpfs/home/pl2948/VariantInterpretation/KNNTestResultArrays/KNN_ESMMAVE_{feature_column[tool]}.npy', test_results_array)
    print("P-KNN completed")

## Plot result

In [ ]:
import numpy as np

feature_column = ['ESM1b_score',
                  'TP53_transcription_urn_mavedb_00001234-0-1_scores',
                  'TP53_DNE_urn_mavedb_00001235-a-1_scores',
                  'urn_mavedb_00000068-b-1_scores',
                  'urn_mavedb_00001234-g-1_scores',
                  'urn_mavedb_00001234-e-1_scores', 
                  'urn_mavedb_00001234-d-1_scores',
                  'urn_mavedb_00001213-a-1_scores',
                  'urn_mavedb_00001236-0-1_scores', 
                  'urn_mavedb_00001234-h-1_scores',
                  'urn_mavedb_00000068-c-1_scores', 
                  'urn_mavedb_00001234-a-1_scores',
                  'urn_mavedb_00001234-c-1_scores', 
                  'urn_mavedb_00001234-b-1_scores',
                  'urn_mavedb_00001234-f-1_scores', 
                  'urn_mavedb_00000068-0-1_scores',
                  'urn_mavedb_00000068-a-1_scores']

feature = TP53_ClinVar[feature_column].to_numpy().astype(float)
print(feature.shape)

label = TP53_ClinVar['ClinVar_annotation'].to_numpy().astype(int)
print(label.shape)

In [ ]:
import torch
from P_KNN_GPU import get_P_KNN_ACMG_score_1D, get_P_KNN_ACMG_score, evaluate_result_1D, evaluate_result, silhouette_score_1d_torch
from sklearn.metrics import roc_auc_score
import numpy as np

p_value = 0.05
Pprior = 0.22
logbase = 1730

combine_data = pd.DataFrame([])
silhouette_score_dict = {}
roc_auc_dict = {}
LLR_single_tool_dict = {}
LLR_P_KNN_dict = {}

for tool in range(len(feature_column)):
    print(f"MAVE: {feature_column[tool]}")
    valid_label = label[~np.isnan(feature[:,tool])]
    valid_feature = feature[~np.isnan(feature[:,tool]), tool]

    silhouette_score= silhouette_score_1d_torch(torch.tensor(valid_feature).reshape(-1, 1), torch.tensor(valid_label))
    print(f"Silhouette score: {silhouette_score:.4f}")
    silhouette_score_dict[feature_column[tool]] = silhouette_score
    
    roc_auc = roc_auc_score(valid_label, valid_feature)
    if roc_auc < 0.5: 
        valid_feature*=-1
        roc_auc = 1-roc_auc
        
    roc_auc_dict[feature_column[tool]] = roc_auc
    
    print("Single-tool")
    test_results_array = np.load(f'/gpfs/home/pl2948/VariantInterpretation/KNNTestResultArrays/KNN_MAVE_{feature_column[tool]}.npy')

    evidence_strength_data, pathogenic_calibration_dict, benign_calibration_dict = evaluate_result_1D(test_results_array,
                                                                                                  valid_feature,
                                                                                                  valid_label, 
                                                                                                  p_value, 
                                                                                                  Pprior, 
                                                                                                  logbase, 
                                                                                                  category = feature_column[tool], 
                                                                                                  show_plot=True, 
                                                                                                  save_name=None)
    
    P_KNN_pathogenic, P_KNN_benign, ACMG_scores = get_P_KNN_ACMG_score_1D(test_results_array, valid_feature, p_value, Pprior, logbase)

    TP53_ClinVar.loc[TP53_ClinVar[feature_column[tool]].notna(), f"ACMGLLR_{feature_column[tool]}"] = ACMG_scores
    
    combine_data = pd.concat([combine_data, evidence_strength_data], ignore_index=True)  
    
    LLR_mean_pathogenic = evidence_strength_data[evidence_strength_data['Label']=='Pathogenic variants']['Score'].mean()
    LLR_mean_benign = evidence_strength_data[evidence_strength_data['Label']=='Benign variants']['Score'].mean()

    LLR_single_tool_dict[feature_column[tool]] = (LLR_mean_pathogenic + LLR_mean_benign)/2
    
    print("pathogenic evidence mean", f"{LLR_mean_pathogenic:.3f}")
    print("benign evidence mean", f"{LLR_mean_benign:.3f}")

    if tool==0: continue

    print("P-KNN")
    test_results_array = np.load(f'/gpfs/home/pl2948/VariantInterpretation/KNNTestResultArrays/KNN_ESMMAVE_{feature_column[tool]}.npy')
    ESM_feature = feature[:, [0, tool]] 
    evidence_strength_data, pathogenic_calibration_dict, benign_calibration_dict = evaluate_result(test_results_array, 
                                                                                               label, 
                                                                                               p_value, 
                                                                                               Pprior, 
                                                                                               logbase, 
                                                                                               category = f"ESM1b_{feature_column[tool]}", 
                                                                                               show_plot=True, 
                                                                                               save_name=tool)

    P_KNN_pathogenic, P_KNN_benign, ACMG_scores = get_P_KNN_ACMG_score(test_results_array, p_value, Pprior, logbase)

    combine_data = pd.concat([combine_data, evidence_strength_data], ignore_index=True) 
    
    LLR_mean_pathogenic = evidence_strength_data[evidence_strength_data['Label']=='Pathogenic variants']['Score'].mean()
    LLR_mean_benign = evidence_strength_data[evidence_strength_data['Label']=='Benign variants']['Score'].mean()

    LLR_P_KNN_dict[feature_column[tool]] = (LLR_mean_pathogenic + LLR_mean_benign)/2
    
    print("pathogenic evidence mean", f"{LLR_mean_pathogenic:.3f}")
    print("benign evidence mean", f"{LLR_mean_benign:.3f}")

# Final figure

In [ ]:
LLR_single_tool_dict

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from adjustText import adjust_text

keys = list(LLR_single_tool_dict.keys())
keys.remove('ESM1b_score')
x1 = np.array([LLR_single_tool_dict[k] for k in keys])
y1 = np.array([LLR_P_KNN_dict[k] for k in keys])
color1 = np.array([silhouette_score_dict[k] for k in keys])
x2 = color1
y2 = y1 / x1
x3 = np.array([roc_auc_dict[k] for k in keys])
y3 = y1 / x1

plt.figure(figsize=(4.7, 6))
scatter1 = plt.scatter(x1, y1, c=color1, cmap='viridis', edgecolor='k')
plt.xlabel('Mean evidence strength (Single Tool)')
plt.ylabel('Mean evidence strength (P-KNN)')
min_val = min(min(x1), min(y1))
max_val = max(max(x1), max(y1))
plt.plot([min_val, max_val], [min_val, max_val], linestyle='--', color='gray', label='y = x')
# plt.legend()
cbar = plt.colorbar(scatter1, orientation='horizontal', pad=0.15)
cbar.set_label('Silhouette Score')
plt.tight_layout()
plt.savefig("Fig6E.svg", format="svg")
plt.show()

plt.figure(figsize=(5, 5))
plt.scatter(x2, y2, c='dodgerblue', edgecolor='k')
plt.axhline(1, color='gray', linestyle='--')
plt.xlabel('Silhouette Score')
plt.ylabel('Mean evidence strength ratio')
plt.tight_layout()
plt.savefig("Fig6F.svg", format="svg")
plt.show()

plt.figure(figsize=(6, 6))
plt.scatter(x3, y3, c='salmon', edgecolor='k')
plt.axhline(1, color='gray', linestyle='--')
plt.xlabel('ROC AUC (Single Tool)')
plt.ylabel('Mean evidence strength ratio (P-KNN / Single)')
plt.tight_layout()
plt.show()


In [ ]:
def evaluate_result(test_results_array, test_label, p_value, Pprior, logbase=None, category = None, show_plot=True, save_name=None):
    if logbase is None:
        logbase = get_logbase(Pprior)
    
    w_test = (1 - Pprior) * sum(test_label == 1) / (sum(test_label == 0) * Pprior)  
    
    # Calculate the pathogenic and benign threshold for evidence level
    Post_p = np.zeros(4) 
    Post_b = np.zeros(4)

    for j in range(4):
        Post_p[j] = logbase ** (1 / 2 ** j) * Pprior / ((logbase ** (1 / 2 ** j) - 1) * Pprior + 1)
        Post_b[j] = (logbase ** (1 / 2 ** j)) * (1 - Pprior) / (((logbase ** (1 / 2 ** j)) - 1) * (1 - Pprior) + 1)

    Pathogenic_test_results_array = test_results_array[test_label == 1]
    Benign_test_results_array = test_results_array[test_label == 0]

    Pathogenic_P_KNN_pathogenic, Pathogenic_P_KNN_benign, Pathogenic_ACMG_scores = get_P_KNN_ACMG_score(
        Pathogenic_test_results_array, p_value, Pprior, logbase)
    
    Benign_P_KNN_pathogenic, Benign_P_KNN_benign, Benign_ACMG_scores = get_P_KNN_ACMG_score(
        Benign_test_results_array, p_value, Pprior, logbase) 
    
    # Print correct and incorrect assignment of pathogenic and benign variants
    print("Pathogenic evidence")
    accumulate_pathogenic_count = 0
    accumulate_benign_count = 0
    for ACMGevidence, threshold in zip(["+8", "+4", "+2", "+1"], Post_p):
        pathogenic_count = (Pathogenic_P_KNN_pathogenic > threshold).sum() - accumulate_pathogenic_count
        benign_count = (Benign_P_KNN_pathogenic > threshold).sum() - accumulate_benign_count
        accumulate_pathogenic_count += pathogenic_count
        accumulate_benign_count += benign_count
        print(f"  Evidence score {ACMGevidence} Probability threshold {threshold:.3f}:")
        print(f"    Pathogenic {pathogenic_count} ({pathogenic_count/(len(Pathogenic_P_KNN_pathogenic)):.2%}) pathogenic variants")
        print(f"    Benign {benign_count} ({benign_count/(len(Benign_P_KNN_pathogenic)):.2%}) error benign variants")
        print(f"    Weighted correct rate: {pathogenic_count/(pathogenic_count+benign_count*w_test):.2%}")

    print("Benign evidence")
    accumulate_pathogenic_count = 0
    accumulate_benign_count = 0
    for ACMGevidence, threshold in zip(["+8", "+4", "+2", "+1"], Post_b):
        pathogenic_count = (Pathogenic_P_KNN_benign > threshold).sum() - accumulate_pathogenic_count
        benign_count = (Benign_P_KNN_benign > threshold).sum() - accumulate_benign_count
        accumulate_pathogenic_count += pathogenic_count
        accumulate_benign_count += benign_count
        print(f"  Evidence score {ACMGevidence} Probability threshold {threshold:.3f}:")
        print(f"    Benign {benign_count} ({benign_count/(len(Benign_P_KNN_benign)):.2%}) benign variants")
        print(f"    Pathogenic {pathogenic_count} ({pathogenic_count/(len(Pathogenic_P_KNN_benign)):.2%}) error pathogenic variants")
        print(f"    Weighted correct rate: {benign_count*w_test/(pathogenic_count+benign_count*w_test):.2%}")

    # Evidence violin plot
    evidence_strength_data = pd.DataFrame({
        "Score": np.concatenate([-Benign_ACMG_scores, Pathogenic_ACMG_scores]),
        "Label": ["Benign"] * len(Benign_ACMG_scores) + ["Pathogenic"] * len(Pathogenic_ACMG_scores),
        "Category": [category] * (len(Benign_ACMG_scores) + len(Pathogenic_ACMG_scores))
        })

    if show_plot:
        import matplotlib.pyplot as plt
        import seaborn as sns
        sns.set(style="ticks")
    
        plt.figure(figsize=(6, 6))
        sns.violinplot(
            x="Category",
            y="Score", 
            hue="Label",   # Pathogenic and Benign 
            data=evidence_strength_data, 
            split=True,   # on each side of violin
            inner="quart", 
            palette={"Pathogenic": "red", "Benign": "blue"},
            alpha=0.6, 
            density_norm='area'
        )
        
        plt.xlabel("")
        plt.ylabel("Evidence Score (log likelihood)", fontsize=14)
        plt.legend(loc='upper right')
        sns.despine(top=True, right=True)
        
        if save_name:
            plt.savefig(f"{save_name}_ACMGevidence.svg", format="svg")
        plt.show()

    # Pathogenic Calibration Plot with Confidence Interval
    num_bins = 10
    bins = np.linspace(0, 1, num_bins + 1)
    
    pathogenic_counts, _ = np.histogram(Pathogenic_P_KNN_pathogenic, bins=bins)
    benign_counts, _ = np.histogram(Benign_P_KNN_pathogenic, bins=bins)

    pathogenic_ratios, ci_lower, ci_upper = weighted_score_with_binom_ci(pathogenic_counts, benign_counts, w_test, p_value)

    bin_centers = (bins[:-1] + bins[1:]) / 2
    valid_mask = ~np.isnan(pathogenic_ratios) & ~np.isnan(ci_lower) & ~np.isnan(ci_upper)

    pathogenic_calibration_dict = {}

    pathogenic_calibration_dict['bin_centers'] = bin_centers[valid_mask]
    pathogenic_calibration_dict['ratios'] = pathogenic_ratios[valid_mask]
    pathogenic_calibration_dict['ci_lower'] = ci_lower[valid_mask]
    pathogenic_calibration_dict['ci_upper'] = ci_upper[valid_mask]


    bins = np.linspace(0.95, 1, num_bins + 1)

    pathogenic_counts, _ = np.histogram(Pathogenic_P_KNN_benign, bins=bins)
    benign_counts, _ = np.histogram(Benign_P_KNN_benign, bins=bins)
    
    benign_ratios, ci_lower, ci_upper = weighted_score_with_binom_ci(benign_counts, pathogenic_counts, 1/w_test, p_value)

    bin_centers = (bins[:-1] + bins[1:]) / 2
    valid_mask = ~np.isnan(benign_ratios) & ~np.isnan(ci_lower) & ~np.isnan(ci_upper)

    benign_calibration_dict = {}
    benign_calibration_dict['bin_centers'] = bin_centers[valid_mask]
    benign_calibration_dict['ratios'] = benign_ratios[valid_mask]
    benign_calibration_dict['ci_lower'] = ci_lower[valid_mask]
    benign_calibration_dict['ci_upper'] = ci_upper[valid_mask]

    if show_plot:
        # Plot the calibration curve for pathogenic
        plt.figure(figsize=(6, 6))
        plt.plot(pathogenic_calibration_dict['bin_centers'], pathogenic_calibration_dict['ratios'], 
                 'o-', color='red', label='Pathogenic Probability')
        plt.fill_between(pathogenic_calibration_dict['bin_centers'], 
                         pathogenic_calibration_dict['ci_lower'], pathogenic_calibration_dict['ci_upper'], 
                         color='red', alpha=0.2, label='95% Confidence Interval')
        plt.xlabel("P-KNN Pathogenic Score", fontsize=14)
        plt.ylabel("Pathogenic Probability", fontsize=14)
        plt.legend(loc='lower right')
        
        x_vals = np.linspace(0, 1, 100)
        plt.plot(x_vals, x_vals, label="y = x", color='red', linestyle='--', alpha=0.2)
        
        for threshold in Post_p:
            plt.axvline(x=threshold, color='orange', linestyle='--', label=f'X = {threshold}', alpha=0.2)
            plt.axhline(y=threshold, color='orange', linestyle='--', label=f'Y = {threshold}', alpha=0.2)

        sns.despine(top=True, right=True)   
        if save_name:
            plt.savefig(f"{save_name}_P_Calibration.svg", format="svg")        
        
        plt.show()

        # Plot the calibration curve for benign
        plt.figure(figsize=(6, 6))
        plt.plot(benign_calibration_dict['bin_centers'], benign_calibration_dict['ratios'],
                 'o-', color='blue', label='Benign Probability')
        plt.fill_between(benign_calibration_dict['bin_centers'],
                         benign_calibration_dict['ci_lower'], benign_calibration_dict['ci_upper'],
                        color='blue', alpha=0.2, label='95% Confidence Interval')
        plt.xlabel("P-KNN Benign Score", fontsize=14)
        plt.ylabel("Benign Probability", fontsize=14)
        plt.ylim(0.95, 1)
        plt.xlim(0.95, 1)
        plt.legend(loc='lower right')
        
        x_vals = np.linspace(0.95, 1, 100)
        plt.plot(x_vals, x_vals, label="y = x", color='blue', linestyle='--', alpha=0.2)
        
        for threshold in Post_b:
            plt.axvline(x=threshold, color='cyan', linestyle='--', label=f'X = {threshold}', alpha=0.2)
            plt.axhline(y=threshold, color='cyan', linestyle='--', label=f'Y = {threshold}', alpha=0.2)

        sns.despine(top=True, right=True)
        if save_name:
            plt.savefig(f"{save_name}_B_Calibration.svg", format="svg")  
            
        plt.show()

    return evidence_strength_data, pathogenic_calibration_dict, benign_calibration_dict

In [ ]:
def get_P_KNN_ACMG_score(test_results_array, p_value, Pprior, logbase=None):
    """
    Calculate the ACMG scores for pathogenicity and benignity based on P-KNN probabilities.
    This function computes the P-KNN pathogenic and benign probabilities, converts them
    to ACMG scores, and adjusts the scores to be within the range [0, 8]. The final ACMG
    scores are calculated as the difference between the pathogenic and benign scores.
    Parameters:
    -----------
    test_results_array : numpy.ndarray
        A 2D array where each row represents the test results for a sample, and each column
        represents a probability value.
    p_value : float
        The p-value used to determine the index for selecting probabilities. Should be in
        the range (0, 1].
    Pprior : float
        The prior probability of pathogenicity.
    logbase : float, optional
        The logarithmic base used for converting probabilities to ACMG scores. If not
        provided, it will be calculated using the `get_logbase` function.
    Returns:
    --------
    P_KNN_pathogenic : numpy.ndarray
        A 1D array containing the P-KNN pathogenic probabilities for each sample.
    P_KNN_benign : numpy.ndarray
        A 1D array containing the P-KNN benign probabilities for each sample.
    ACMG_scores : numpy.ndarray
        A 1D array containing the ACMG scores for each sample, calculated as the
        difference between the pathogenic and benign scores.
    Notes:
    ------
    - The ACMG scores are clamped to the range [-8, 8].
    - The input `test_results_array` is expected to have at least one column, and the
      `p_value` should be chosen such that the calculated index is valid.
    """

    index = int(np.ceil(p_value*test_results_array.shape[1]))
    P_KNN_pathogenic = np.sort(test_results_array, axis=1)[:, index-1]
    P_KNN_benign = 1- np.sort(test_results_array, axis=1)[:, -index]

    if logbase is None:
        logbase = get_logbase(Pprior) 

    ACMG_pathogenic_scores = Probability2ACMG_score(P_KNN_pathogenic, Pprior, logbase)
    ACMG_benign_scores = Probability2ACMG_score(P_KNN_benign, 1-Pprior, logbase)

    ACMG_pathogenic_scores[ACMG_pathogenic_scores < 0] = 0
    ACMG_benign_scores[ACMG_benign_scores < 0] = 0
    ACMG_pathogenic_scores[ACMG_pathogenic_scores > 8] = 8
    ACMG_benign_scores[ACMG_benign_scores > 8] = 8

    ACMG_scores = ACMG_pathogenic_scores - ACMG_benign_scores

    return P_KNN_pathogenic, P_KNN_benign, ACMG_scores

def Probability2ACMG_score(Ppost, Pprior, logbase=None):
    if logbase is None:
        logbase = get_logbase(Pprior)

    Likelihood_ratio = Ppost * (1 - Pprior) / ((1 - Ppost) * Pprior)
    ACMG_scores = 8 * np.log(Likelihood_ratio) / np.log(logbase)

    return ACMG_scores


def ACMG_score2Probability(ACMG_scores, Pprior, logbase=None):
    if logbase is None:
        logbase = get_logbase(Pprior)
    
    Pathogenic_likelihood = logbase ** (ACMG_scores / 8)
    Pathogenic_prob = Pathogenic_likelihood * Pprior / (1 + Pprior * (Pathogenic_likelihood-1)) 
    return Pathogenic_prob

def weighted_score_with_binom_ci(p_array, n_array, w, p_value=0.05):
    from scipy.stats import binom
    p_array = np.asarray(p_array)
    n_array = np.asarray(n_array)
    shape = p_array.shape

    scores = np.full(shape, np.nan)
    ci_lower = np.full(shape, np.nan)
    ci_upper = np.full(shape, np.nan)

    t_array = p_array + n_array

    it = np.nditer(p_array, flags=['multi_index'])
    while not it.finished:
        idx = it.multi_index
        p = p_array[idx]
        t = t_array[idx]

        if t > 0:
            pi_hat = p / t
            ci_low, ci_up = binom.interval(1-p_value, t, pi_hat)
            
            def weighted(pi):
                return pi / (pi + w * (1 - pi))

            scores[idx] = weighted(pi_hat)
            ci_lower[idx] = weighted(ci_low / t)
            ci_upper[idx] = weighted(ci_up / t)
        
        it.iternext()
    
    return scores, ci_lower, ci_upper


In [ ]:
TP53_ClinVar.columns

In [ ]:
TP53_ClinVar['ACMGLLR_DMS_trp+ESM'] = TP53_ClinVar['ACMGLLR_TP53_transcription_urn_mavedb_00001234-0-1_scores']+TP53_ClinVar['ACMGLLR_ESM1b_score']
TP53_ClinVar['ACMGLLR_DMS_dne+ESM'] = TP53_ClinVar['ACMGLLR_TP53_DNE_urn_mavedb_00001235-a-1_scores']+TP53_ClinVar['ACMGLLR_ESM1b_score']
# TP53_ClinVar[(#(TP53_ClinVar['ACMGLLR_DMS_trp']<TP53_ClinVar['ACMGLLR_DMS_trp_ESM']) & \
#               (TP53_ClinVar['ClinVar_annotation']==1))][['ACMGLLR_DMS_trp', 'ACMGLLR_DMS_trp_ESM']]
TP53_ClinVar[(TP53_ClinVar['ClinVar_annotation']==1)][['ACMGLLR_ESM1b_score', 'ACMGLLR_TP53_DNE_urn_mavedb_00001235-a-1_scores', 'ACMGLLR_DMS_dne+ESM']].describe()

In [ ]:
combine_data = combine_data[combine_data['Category'].isin(['ESM1b_score', 
                                                            'TP53_transcription_urn_mavedb_00001234-0-1_scores',
                                                            'ESM1b_TP53_transcription_urn_mavedb_00001234-0-1_scores',
                                                            'TP53_DNE_urn_mavedb_00001235-a-1_scores', 
                                                            'ESM1b_TP53_DNE_urn_mavedb_00001235-a-1_scores'])]

combine_data['Category'] = combine_data['Category'].replace({
    'ESM1b_score': 'ESM1b_score',
    'TP53_transcription_urn_mavedb_00001234-0-1_scores': 'DMS_trp',
    'ESM1b_TP53_transcription_urn_mavedb_00001234-0-1_scores': 'DMS_trp_ESM',
    'TP53_DNE_urn_mavedb_00001235-a-1_scores': 'DMS_dne',
    'ESM1b_TP53_DNE_urn_mavedb_00001235-a-1_scores': 'DMS_dne_ESM'
})

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt 

## violin plot
sns.set(style="ticks")
plt.figure(figsize=(12, 3.5))
ax = sns.violinplot(
    x="Category",
    y="Score", 
    hue="Label",   # Pathogenic and Benign 
    data=combine_data, 
    split=True,   # on each side of violin
    inner="quart", 
    palette={"Pathogenic variants": "red", "Benign variants": "blue"},
    alpha=0.6, 
    density_norm='count',
    hue_order=['Pathogenic variants', 'Benign variants'],
    order=['ESM1b_score', 'DMS_dne', 'DMS_dne_ESM', 'DMS_trp', 'DMS_trp_ESM']
)

dark_gray = '0.3'

sns.boxplot(
    x="Category", y="Score", hue="Label", data=combine_data,
    showcaps=True,  
    showfliers=False,
    palette={"Pathogenic variants": "pink", "Benign variants": "#bae0f5"},
    width=0.1, dodge=True, ax=ax,
    whiskerprops={'color': dark_gray, 'linewidth': 1.5, 'zorder': 2},
    capprops={'color': dark_gray, 'linewidth': 1.5},
    medianprops={'color': dark_gray, 'linewidth': 1.5},
    boxprops={'zorder': 2, 'edgecolor': dark_gray, 'linewidth': 1.5},
    hue_order=['Pathogenic variants', 'Benign variants']

)

handles, labels = ax.get_legend_handles_labels()
n_hue = combine_data["Label"].nunique() 
ax.legend(handles[:n_hue], labels[:n_hue], loc="lower right", fontsize=12)
plt.xlabel("")
plt.xticks(fontsize=14)
plt.yticks([-8, -6, -4, -2, 0, 2, 4, 6, 8])
# plt.ylim([-6,11])
plt.ylabel("Evidence strength (LLR)", fontsize=14)
sns.despine(top=True, right=True)

plt.savefig("Fig6A.svg", format="svg")
plt.show()

In [ ]:
num_bins = 10

Pprior = 0.22
label = TP53_ClinVar['ClinVar_annotation'].to_numpy().astype(int)
w_test = (1 - Pprior) * sum(label == 1) / (sum(label == 0) * Pprior) 

Ppost = ACMG_score2Probability(TP53_ClinVar['ACMGLLR_DMS_dne+ESM'], Pprior, logbase=1730)

Post_p = np.zeros(4) 
Post_b = np.zeros(4)

for j in range(4):
    Post_p[j] = logbase ** (1 / 2 ** j) * Pprior / ((logbase ** (1 / 2 ** j) - 1) * Pprior + 1)
    Post_b[j] = (logbase ** (1 / 2 ** j)) * (1 - Pprior) / (((logbase ** (1 / 2 ** j)) - 1) * (1 - Pprior) + 1)
        
Pathogenic_P_KNN_pathogenic = Ppost[label==1]
Benign_P_KNN_pathogenic = Ppost[label==0]

bins = np.linspace(0, 1, num_bins + 1)

pathogenic_counts, _ = np.histogram(Pathogenic_P_KNN_pathogenic, bins=bins)
benign_counts, _ = np.histogram(Benign_P_KNN_pathogenic, bins=bins)

pathogenic_ratios, ci_lower, ci_upper = weighted_score_with_binom_ci(pathogenic_counts, benign_counts, w_test, p_value)

bin_centers = (bins[:-1] + bins[1:]) / 2
valid_mask = ~np.isnan(pathogenic_ratios) & ~np.isnan(ci_lower) & ~np.isnan(ci_upper)

pathogenic_calibration_dict = {}

pathogenic_calibration_dict['bin_centers'] = bin_centers[valid_mask]
pathogenic_calibration_dict['ratios'] = pathogenic_ratios[valid_mask]
pathogenic_calibration_dict['ci_lower'] = ci_lower[valid_mask]
pathogenic_calibration_dict['ci_upper'] = ci_upper[valid_mask]

plt.figure(figsize=(6, 6))
plt.plot(pathogenic_calibration_dict['bin_centers'], pathogenic_calibration_dict['ratios'], 
         'o-', color='red', label='Pathogenic Probability')
plt.fill_between(pathogenic_calibration_dict['bin_centers'], 
                 pathogenic_calibration_dict['ci_lower'], pathogenic_calibration_dict['ci_upper'], 
                 color='red', alpha=0.2, label='95% Confidence Interval')
plt.xlabel("P-KNN Pathogenic Score", fontsize=14)
plt.ylabel("Pathogenic Probability", fontsize=14)
plt.legend(loc='lower right')

x_vals = np.linspace(0, 1, 100)
plt.plot(x_vals, x_vals, label="y = x", color='red', linestyle='--', alpha=0.2)

for threshold in Post_p:
    plt.axvline(x=threshold, color='orange', linestyle='--', label=f'X = {threshold}', alpha=0.2)
    plt.axhline(y=threshold, color='orange', linestyle='--', label=f'Y = {threshold}', alpha=0.2)

sns.despine(top=True, right=True)
plt.savefig(f"Fig6E_P_Calibration.svg", format="svg")        

plt.show()